In [1]:
import numpy as np
import scipy as sp
from scipy import sparse as sparse
from scipy.sparse import linalg as ln

In [2]:
n_points = 1000
x_begin = -200.0
x_end = 200.0
x0 = -150.0
k0 = 1.0
barrier_height = 1.0
barrier_width = 3.0
sigma0 = 5.0
dt = 0.5

In [3]:
x, dx = np.linspace(x_begin, x_end, n_points, retstep=True)

In [4]:
potential = np.array([barrier_height if 0.0<x1<barrier_width else 0.0 for x1 in x])
potential.size

1000

In [5]:
h_diag = np.ones(n_points)/dx**2 + potential
h_diag

array([6.23750625, 6.23750625, 6.23750625, 6.23750625, 6.23750625,
       6.23750625, 6.23750625, 6.23750625, 6.23750625, 6.23750625,
       6.23750625, 6.23750625, 6.23750625, 6.23750625, 6.23750625,
       6.23750625, 6.23750625, 6.23750625, 6.23750625, 6.23750625,
       6.23750625, 6.23750625, 6.23750625, 6.23750625, 6.23750625,
       6.23750625, 6.23750625, 6.23750625, 6.23750625, 6.23750625,
       6.23750625, 6.23750625, 6.23750625, 6.23750625, 6.23750625,
       6.23750625, 6.23750625, 6.23750625, 6.23750625, 6.23750625,
       6.23750625, 6.23750625, 6.23750625, 6.23750625, 6.23750625,
       6.23750625, 6.23750625, 6.23750625, 6.23750625, 6.23750625,
       6.23750625, 6.23750625, 6.23750625, 6.23750625, 6.23750625,
       6.23750625, 6.23750625, 6.23750625, 6.23750625, 6.23750625,
       6.23750625, 6.23750625, 6.23750625, 6.23750625, 6.23750625,
       6.23750625, 6.23750625, 6.23750625, 6.23750625, 6.23750625,
       6.23750625, 6.23750625, 6.23750625, 6.23750625, 6.23750

In [6]:
h_non_diag = np.ones(n_points-1)*(-0.5)/dx**2
h_non_diag

array([-3.11875312, -3.11875312, -3.11875312, -3.11875312, -3.11875312,
       -3.11875312, -3.11875312, -3.11875312, -3.11875312, -3.11875312,
       -3.11875312, -3.11875312, -3.11875312, -3.11875312, -3.11875312,
       -3.11875312, -3.11875312, -3.11875312, -3.11875312, -3.11875312,
       -3.11875312, -3.11875312, -3.11875312, -3.11875312, -3.11875312,
       -3.11875312, -3.11875312, -3.11875312, -3.11875312, -3.11875312,
       -3.11875312, -3.11875312, -3.11875312, -3.11875312, -3.11875312,
       -3.11875312, -3.11875312, -3.11875312, -3.11875312, -3.11875312,
       -3.11875312, -3.11875312, -3.11875312, -3.11875312, -3.11875312,
       -3.11875312, -3.11875312, -3.11875312, -3.11875312, -3.11875312,
       -3.11875312, -3.11875312, -3.11875312, -3.11875312, -3.11875312,
       -3.11875312, -3.11875312, -3.11875312, -3.11875312, -3.11875312,
       -3.11875312, -3.11875312, -3.11875312, -3.11875312, -3.11875312,
       -3.11875312, -3.11875312, -3.11875312, -3.11875312, -3.11

In [7]:
hamiltonian = sparse.diags([h_diag,h_non_diag,h_non_diag],[0,1,-1])
hamiltonian

<1000x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 2998 stored elements (3 diagonals) in DIAgonal format>

In [8]:
norm = (2*np.pi*sigma0**2)**(-0.25)

In [9]:
psi = np.exp(-((x-x0)/2/sigma0)**2)
psi_mom = np.exp(1.0j*k0*x)
psi = psi*psi_mom

In [10]:
momentum = np.exp(1.0j*k0*x)
momentum*x

array([-9.74375350e+01-1.74659459e+02j, -2.16073632e+01-1.98426616e+02j,
        5.73305701e+01-1.90770875e+02j,  1.26900445e+02-1.53026924e+02j,
        1.76155665e+02-9.12748939e+01j,  1.97401215e+02-1.53612353e+01j,
        1.87395649e+02+6.26712163e+01j,  1.47844965e+02+1.30493682e+02j,
        8.51132420e+01+1.77439328e+02j,  9.19889379e+00+1.96180847e+02j,
       -6.78588283e+01+1.83873897e+02j, -1.33889126e+02+1.42588005e+02j,
       -1.78511952e+02+7.89597826e+01j, -1.94769705e+02+3.12637525e+00j,
       -1.80211827e+02-7.28894918e+01j, -1.37263279e+02-1.37085601e+02j,
       -7.28216265e+01-1.79375278e+02j,  2.85046263e+00-1.93172164e+02j,
        7.77595292e+01-1.76415749e+02j,  1.40082189e+02-1.31878023e+02j,
        1.80031284e+02-6.67057846e+01j,  1.91392779e+02+8.72594653e+00j,
        1.72492065e+02+8.24655022e+01j,  1.26439465e+02+1.42878226e+02j,
        6.06191600e+01+1.80482182e+02j, -1.44945929e+01+1.89436277e+02j,
       -8.70042139e+01+1.68447260e+02j, -1.45473305

In [14]:
#computing the Crank-Nicholson Algorithm
implicit = (sparse.eye(n_points)-dt/(2.0j)*hamiltonian).tocsc()
explicit = (sparse.eye(n_points)+dt/2.0j*hamiltonian).tocsc()
evolution_matrix = ln.inv(implicit).dot(explicit).tocsr()

In [15]:
evolution_matrix

<1000x1000 sparse matrix of type '<class 'numpy.complex128'>'
	with 988225 stored elements in Compressed Sparse Row format>